# Week Five Part 2

Ari and Lucas

Our assignment for Week 5 is to use a dataset to predict the class of new documents as either spam and ham (non-spam) e-mails.

We will be using the dataset provided for us:  UCI Machine Learning Repository: Spambase Data Set (https://archive.ics.uci.edu/dataset/94/spambase). Our first step is to load the data and do some prints on it just to see what we are working with.

In [11]:
import pandas as pd

#prepare column names
column_names = [
    'word_freq_make', 'word_freq_address', 'word_freq_all', 'word_freq_3d',
    'word_freq_our', 'word_freq_over', 'word_freq_remove', 'word_freq_internet',
    'word_freq_order', 'word_freq_mail', 'word_freq_receive', 'word_freq_will',
    'word_freq_people', 'word_freq_report', 'word_freq_addresses', 'word_freq_free',
    'word_freq_business', 'word_freq_email', 'word_freq_you', 'word_freq_credit',
    'word_freq_your', 'word_freq_font', 'word_freq_000', 'word_freq_money',
    'word_freq_hp', 'word_freq_hpl', 'word_freq_george', 'word_freq_650',
    'word_freq_lab', 'word_freq_labs', 'word_freq_telnet', 'word_freq_857',
    'word_freq_data', 'word_freq_415', 'word_freq_85', 'word_freq_technology',
    'word_freq_1999', 'word_freq_parts', 'word_freq_pm', 'word_freq_direct',
    'word_freq_cs', 'word_freq_meeting', 'word_freq_original', 'word_freq_project',
    'word_freq_re', 'word_freq_edu', 'word_freq_table', 'word_freq_conference',
    'char_freq_;', 'char_freq_(', 'char_freq_[', 'char_freq_!', 'char_freq_$',
    'char_freq_#', 'capital_run_length_average', 'capital_run_length_longest',
    'capital_run_length_total', 'target'
]

#load data
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data"
spambase = pd.read_csv(url, header=None, names=column_names)

#prints
print(spambase.head())
print(f"Total emails/rows: {len(spambase)}")
print("Total spam emails (target=1):", spambase['target'].sum())
print("Total ham emails (target=0):", len(spambase) - spambase['target'].sum())

   word_freq_make  word_freq_address  word_freq_all  word_freq_3d  \
0            0.00               0.64           0.64           0.0   
1            0.21               0.28           0.50           0.0   
2            0.06               0.00           0.71           0.0   
3            0.00               0.00           0.00           0.0   
4            0.00               0.00           0.00           0.0   

   word_freq_our  word_freq_over  word_freq_remove  word_freq_internet  \
0           0.32            0.00              0.00                0.00   
1           0.14            0.28              0.21                0.07   
2           1.23            0.19              0.19                0.12   
3           0.63            0.00              0.31                0.63   
4           0.63            0.00              0.31                0.63   

   word_freq_order  word_freq_mail  ...  char_freq_;  char_freq_(  \
0             0.00            0.00  ...         0.00        0.000   
1 

There is 58 columns in spambase with a most of them holding relative frequencies of different words and characters. The couple extras are on capital letters as a lot of spam emails often have a lot of these i.e. URGENT NOTICE or LIMITED TIME OFFER. The last column 'target' is the binary column for spam (1) or ham (0).

Next, we need to split our dataset into training and testing sets. We can see that the spambase has 4,601 rows of email info. A good split for training and testing sets is between 70:30 to 80:20. In this case, we will use 70:30.

In [98]:
from sklearn.model_selection import train_test_split

# Split: 70% train, 30% test
X_train, X_test, y_train, y_test = train_test_split(spambase, spambase['target'], test_size=0.3, random_state=77)

print(f"Training set: {len(X_train)}")
print(f"Testing set: {len(X_test)}")

Training set: 3220
Testing set: 1381


Next, we need to train a classifier for predicting if a email is spam or not. We chose Logistic Regression as our classifier because it is good with binary predictions and works well with numeric values which all our columns are.

Below is a simple Logistic Regression classifier with just capital_run_length_total as our only predicting feature. This columns tell us the total number of capital letters in an email. As discussed above, a lot of spam emails tend to use capital letters to make their emails look urgent or pop out as the viewer.

In [99]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

#Logistic Regression model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train[['capital_run_length_total']], y_train)

#use Logistic Regression model to make predictions on the test set
results = model.predict(X_test[['capital_run_length_total']])

#print results
print(classification_report(y_test, results))

accuracy = accuracy_score(y_test, results)
print(f"Accuracy: {accuracy * 100:.2f}%")


              precision    recall  f1-score   support

           0       0.63      0.92      0.75       803
           1       0.70      0.25      0.37       578

    accuracy                           0.64      1381
   macro avg       0.66      0.59      0.56      1381
weighted avg       0.66      0.64      0.59      1381

Accuracy: 64.08%


The above simple Logistic Regression classifier got an accuracy score of 64.08% which is not that good considering this is a binary prediction so the average if we did this randomly with no features is 50%. This improve this we can add more features and we can standardize our features by scaling them after picking which to use.

The features we added (w/ capital_run_length_total):
*   the remaining 2 about capital letters:
  *   capital_run_length_average (average length of capital letter runs in an email i.e. 2,4,3 -> 3)
  *   capital_run_length_longest (longest capital letter run in an email i.e. 2,4,3 -> 4)
*   char_freq_: the frequency of the specific character in the email
  *   '!'
  *   '$'
  *   '#'
*   word_freq_: the frequency of the specific word in the email

We added char and word frequency features for those that typically pop up in spam emails. Initially based on personal experience, we decided to choose our word_freq_ columns on the ones that come up the most in the spam emails we have here. Below is code to take all the spam emails from our training set, find the averages for each word_freq_ column, sort them, and print the top 20.

In [100]:
#get only spam emails in the training set
spam_only = X_train[y_train == 1]

#calculate the avg frequency of each word_freq column in these spam emails
avg_spam_word_freq = spam_only[[col for col in X_train.columns if col.startswith('word_freq_')]].mean()

#sort avg_spam_word_freq - descending
sorted_avg_freq = avg_spam_word_freq.sort_values(ascending=False)

#print
print("Top 20 most frequent words in spam emails:\n")
for word, freq in sorted_avg_freq.head(20).items():
    print(f"{word:<20} {freq:.4f}")


Top 20 most frequent words in spam emails:

word_freq_you        2.2610
word_freq_your       1.4190
word_freq_will       0.5408
word_freq_our        0.5361
word_freq_free       0.5135
word_freq_all        0.4095
word_freq_mail       0.3558
word_freq_email      0.3134
word_freq_remove     0.2922
word_freq_business   0.2803
word_freq_000        0.2487
word_freq_credit     0.2343
word_freq_money      0.2181
word_freq_internet   0.2176
word_freq_font       0.2164
word_freq_over       0.1757
word_freq_order      0.1747
word_freq_3d         0.1574
word_freq_address    0.1561
word_freq_make       0.1439


This gives us insight on what our data in training set looks like. We do need to keep in mind that some of these top 20 are due to it being a function words or common words in emails so we need to filter through this list.

In addition to the features we added above, we will also be adding word_freq_ columns for:
*   you
*   your
*   free
*   remove
*   business

In [101]:
from sklearn.preprocessing import StandardScaler

#create list for the features as there is more than one now
selected_features = [
    'capital_run_length_total',
    #'capital_run_length_average', - was kicked out during testing
    'capital_run_length_longest',
    'char_freq_!',
    'char_freq_$',
    'char_freq_#',
    'word_freq_you',
    'word_freq_your',
    'word_freq_free',
    'word_freq_remove',
    'word_freq_business'
]
X_train_feature = X_train[selected_features]
X_test_feature = X_test[selected_features]

#scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_feature)
X_test_scaled = scaler.transform(X_test_feature)

#Logistic Regression model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train)

#use Logistic Regression model to make predictions on the test set
results = model.predict(X_test_scaled)

#print results
print(classification_report(y_test, results))

accuracy = accuracy_score(y_test, results)
print(f"Accuracy: {accuracy * 100:.2f}%")

              precision    recall  f1-score   support

           0       0.83      0.95      0.89       803
           1       0.91      0.74      0.81       578

    accuracy                           0.86      1381
   macro avg       0.87      0.84      0.85      1381
weighted avg       0.86      0.86      0.86      1381

Accuracy: 85.88%


With the new additional features and scaling, the accuracy went up from 64.08% to 85.88%. That's a pretty good increase and after testing different sets of features, we are happy with that!

This was a very interesting look into spam emails and how email system determine what is spam or ham. We would have liked to add our own spam emails to this but with the time constraints, we could not. Maybe in the next steps.

It would have been cool if we had data on the email addresses these emails were sent from too. That way we can get columns for if this email is in this person's contacts, what kind of email system they are using (gmail, hotmail, outlook and .com, .org, .edu), and if it is misspelled? That one might be hard to do.

Either way, there is more work that can be done to improve this prediction accuracy if we have more information available.